<a href="https://colab.research.google.com/github/williamvpaiva/MercadoFinanceiro/blob/main/FIIS_VERS%C3%83O_FINAL_Analise_de_Fundos_Imobiliarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 9.3 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=29b85850de1185ce7f650ed046830f407bbfe4c1160f37abe1fa6ce3810a2a08
  Stored in directory: /tmp/pip-ephem-wheel-cache-figx0z78/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
from datetime import datetime
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly
import openpyxl
import numpy as np


In [3]:
url ='https://www.fundsexplorer.com.br/ranking'

In [4]:
agent = {"User-Agent":"Mozilla/5.0"}
response = requests.get(url)
soup = BeautifulSoup(response.text,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table),decimal=',',thousands='.')[0]
df

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (3M)Média,DY (6M)Média,DY (12M)Média,DY Ano,Variação Preço,Rentab.Período,Rentab.Acumulada,PatrimônioLíq.,VPA,P/VPA,DYPatrimonial,VariaçãoPatrimonial,Rentab. Patr.no Período,Rentab. Patr.Acumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
0,FIVN11,Shoppings,"R$ 2,84",83390.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%",NaN,"0,00%","0,00%","0,00%","R$ 64.722.802,83","R$ 6,88",0.41,NaN,NaN,NaN,NaN,"56,00%",NaN,1
1,BZLI11,Títulos e Val. Mob.,"R$ 14,70",440410.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,68%","0,68%","1,14%","R$ 444.984.851,42","R$ 11,37",1.29,NaN,NaN,NaN,NaN,NaN,NaN,0
2,XTED11,Lajes Corporativas,"R$ 6,87",820.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","1,10%","1,10%","-37,83%","R$ 25.791.388,88","R$ 13,93",0.49,NaN,NaN,NaN,NaN,"0,00%","100,00%",1
3,ALMI11,Lajes Corporativas,"R$ 1.155,00",190.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","2,27%","2,27%","-28,29%","R$ 229.163.662,27","R$ 2.061,25",0.56,NaN,NaN,NaN,NaN,"68,15%",NaN,1
4,DOMC11,Lajes Corporativas,"R$ 439,00",420.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","4,35%","4,35%","-10,00%","R$ 269.460.438,30","R$ 110,26",3.98,NaN,NaN,NaN,NaN,"23,44%",NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,RBRP11,Outros,"R$ 79,80",203280.0,"R$ 0,42","0,53%","1,33%","2,78%","6,34%","0,44%","0,46%","0,53%","3,31%","-0,75%","-0,22%","-16,00%","R$ 1.042.431.997,69","R$ 85,59",0.93,NaN,NaN,NaN,NaN,"6,00%","4,00%",17
246,FLCR11,Títulos e Val. Mob.,"R$ 100,62",3350.0,"R$ 0,97","0,96%","0,00%","0,00%","0,00%","0,00%","0,00%","0,00%","0,96%","0,00%","0,96%","0,96%","R$ 50.417.226,04","R$ 100,83",1.00,NaN,NaN,NaN,NaN,NaN,NaN,0
247,RECT11,Híbrido,"R$ 79,90",183620.0,"R$ 0,50","0,63%","2,13%","4,39%","8,85%","0,71%","0,73%","0,74%","5,14%","1,45%","2,08%","-12,46%","R$ 761.018.357,05","R$ 93,04",0.86,NaN,NaN,NaN,NaN,"33,20%",NaN,8
248,RELG11,Híbrido,"R$ 87,59",63370.0,"R$ 0,75","0,86%","2,34%","4,65%","0,00%","0,78%","0,77%","0,00%","5,29%","-4,62%","-3,80%","-6,07%","R$ 128.733.540,44","R$ 96,46",0.91,NaN,NaN,NaN,NaN,"15,50%","15,50%",3


In [5]:
#df['DividendYield'].dropna(inplace=True) 
url

'https://www.fundsexplorer.com.br/ranking'

In [6]:
df = df.rename(columns={'Códigodo fundo':'Codigofundo',
                        #'Preço Atual':'Preco',
                        'Liquidez Diária':'LiqDia',
                       # 'Dividendo':'Dividendo',
                        #'DividendYield':'DY'
                        'DY (12M)Média':'DY12MM',
                        #'DY (3M)Acumulado':'(DY_3M)',
                        #'DY (6M)Acumulado':'DY_(6M)',
                        #'DY (12M)Acumulado':'DY(12M)',
                       # 'DY (3M)Média':'DY(3M)Media',
                        #'DY (6M)Média':'DY(6M)Media',
                        'DY Ano':'DYAno',
                        'PatrimônioLíq.':'Patrimonio',
                        'P/VPA':'PVPA',
                        'Rentab.Acumulada':'RentabAcumulada'
                        #'QuantidadeAtivos': 'ativos'
#, 'PatrimônioLíq.':'Patr Liqu'
})

In [7]:
df['DividendYield'] = df['DividendYield'].str.replace(r'\D', '').astype(float)/100
df['DYAno'] = df['DYAno'].str.replace(r'\D', '').astype(float)/100
df['DY12MM'] = df['DY12MM'].str.replace(r'\D', '').astype(float)/100
df['RentabAcumulada'] = df['RentabAcumulada'].str.replace(r'\D', '').astype(float)/100
df['VacânciaFísica'] = df['VacânciaFísica'].str.replace(r'\D', '').astype(float)/100
df['Patrimonio'] = df['Patrimonio'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("R$","")))
df['Dividendo'] = df['Dividendo'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("R$","")))
#df['Preco'] = df['Preco'].apply(lambda x: float(x.replace(".","").replace(",",".")))

In [8]:
#df['Preco'] = df['Preco'].apply(lambda x: float(x.replace("R$","").replace(",",".")))

In [9]:
#df

In [10]:
 #df = df.query('LiqDia >= 200000')

In [11]:
 #df = df.query('Patrimonio >= 300000000')

In [12]:
# Filtros 
#df =  df.query('DividendYield >= 0.45')

In [13]:
#df = df.query('DY12MM >= 0.40')

In [14]:
#df = df.query('PVPA <= 1.10') 

In [15]:
df = df.query("Codigofundo != 'GTWR11' and Codigofundo != 'XPPR11'  and Setor != 'Residencial' and DY12MM >= 0.4  and DividendYield >= 0.4 ")  
 
df = df.query("LiqDia >= 100000")
 
# Price vs net worth
df['acceptable_pricing'] = np.where((df['PVPA'] > 0.7) & (df['PVPA'] < 1.1) , True, False)
 
 
# Asset diversification
diverssified_assets_conditions = [
  #  # 1st Condition
    (df['Setor'].isin(['Shoppings', 'Lajes Corporativas', 'Logistica'])) &
    (df['QuantidadeAtivos'] > 5),
    # 2st Condition
    (df['Setor'].isin(['Shoppings', 'Lajes Corporativas', 'Logistica'])) &
    (df['QuantidadeAtivos'] < 5),
#    # 3rd Condition
    (~df['Setor'].isin(['Shoppings', 'Lajes Corporativas', 'Logistica']))
]
diverssified_assets_choices = [True, False, True]
df['diverssified_assets'] = np.select(diverssified_assets_conditions, diverssified_assets_choices, default=None)
 
# Vacancy
df['razonably_occupied'] = np.where((df['VacânciaFísica'] < 10.00) | (df['VacânciaFísica'].isnull()) | (df['VacânciaFísica'].isnull()) , True, False)
 
# Liquidity
#df['is_lLiqDiaiquid'] = np.where(df['LiqDia'] > 200000, True, False)
 
# Patrimonio 
df['Patrimonio'] = np.where(df['Patrimonio'] > 500000000, True, False)
 
 
#df['DY12MM'] = np.where(df['DY (12M)Acumulado'] > 'mean', True, False)
 

 
# Patrimonio 
#df['Codigofundo'] = np.where(df['Codigofundo']  != 'GTWR11', True, False)
 
#f = df.query("LiqDia >= 200000 and Patrimonio >= 800000000 and Codigofundo != 'GTWR11'
 
# dividend yield
 
 
#VacânciaFísica VacânciaFinanceira

In [16]:
#df =  df.query("Codigofundo != ´Títulos e Val. Mob.´")
#df = df.query("LiqDia >= 200000 and Setor != 'Títulos e Val. Mob.' and PVPA <= 1.10 and DY12MM >= 0.40 and DividendYield >= 0.45 and Patrimonio >= 500000000") 
#df = df.query("LiqDia >= 200000 and Setor != 'Títulos e Val. Mob.' and PVPA <= 1.10 and Patrimonio >= 900000000") 
#df = df.query("LiqDia >= 200000 and PVPA <= 1.10 and Patrimonio >= 800000000 and Codigofundo != 'GTWR11' and RentabAcumulada > 0.00  and PVPA > 0.9 & PVPA < 1.1 ") 
#df = df.query("LiqDia >= 200000 and Patrimonio >= 800000000 and Codigofundo != 'GTWR11' and RentabAcumulada > 0.00  and PVPA > 0.9 & PVPA < 1.1 ") 
#df = df.query("Codigofundo != 'GTWR11' and Setor != 'Residencial' and DY12MM >= 0.03  and DividendYield >= 0.03")  
#df = df.query("LiqDia >= 200000 and PVPA <= 1.10 and Patrimonio >= 500000000 and Codigofundo != 'GTWR11' and RentabAcumulada > 0.00  and PVPA > 0.7 & PVPA < 1.1 and DY12MM >= 0.00 and DividendYield >= 0.00")

In [17]:
df

,Codigofundo,Setor,Preço Atual,LiqDia,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (3M)Média,DY (6M)Média,DY12MM,DYAno,Variação Preço,Rentab.Período,RentabAcumulada,Patrimonio,VPA,PVPA,DYPatrimonial,VariaçãoPatrimonial,Rentab. Patr.no Período,Rentab. Patr.Acumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos,acceptable_pricing,diverssified_assets,razonably_occupied
37,CPFF11,Híbrido,"R$ 80,50",153960.0,0.80,0.98,"2,93%","5,74%","9,16%","0,98%","0,96%",0.76,5.74,"-3,24%","-2,29%",1.32,False,"R$ 80,68",1.00,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True
39,CVBI11,Títulos e Val. Mob.,"R$ 103,80",246300.0,1.00,0.95,"2,85%","5,93%","12,40%","0,95%","0,99%",1.03,5.93,"0,10%","1,05%",2.47,True,"R$ 99,94",1.04,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True
41,IRDM11,Títulos e Val. Mob.,"R$ 118,42",627740.0,0.97,0.75,"2,45%","5,16%","10,83%","0,82%","0,86%",0.90,5.16,"-1,30%","-0,56%",6.89,True,"R$ 97,26",1.22,NaN,NaN,NaN,NaN,NaN,NaN,0,False,True,True
42,RBRF11,Títulos e Val. Mob.,"R$ 80,50",269240.0,0.60,0.74,"2,07%","3,97%","8,21%","0,69%","0,66%",0.68,3.97,"-7,89%","-7,20%",15.11,True,"R$ 88,04",0.91,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True
44,CPTS11,Títulos e Val. Mob.,"R$ 99,80",830660.0,1.00,0.99,"3,02%","6,06%","11,29%","1,01%","1,01%",0.94,6.06,"0,15%","1,14%",4.31,True,"R$ 95,35",1.05,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True
45,VGIP11,Outros,"R$ 101,53",107080.0,0.95,0.93,"2,79%","5,61%","11,52%","0,93%","0,93%",0.96,5.61,"-3,32%","-2,42%",5.67,True,"R$ 95,68",1.06,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True
48,MCCI11,Títulos e Val. Mob.,"R$ 102,30",258120.0,1.00,0.97,"2,53%","4,56%","8,08%","0,84%","0,76%",0.67,4.56,"0,00%","0,97%",4.60,True,"R$ 100,29",1.02,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True
93,TRXF11,Outros,"R$ 102,48",144380.0,0.70,0.68,"2,02%","3,95%","7,46%","0,67%","0,66%",0.62,4.60,"-0,24%","0,43%",1.40,True,"R$ 106,31",0.96,NaN,NaN,NaN,NaN,0.00,NaN,44,True,True,True
115,HFOF11,Títulos e Val. Mob.,"R$ 91,75",178560.0,0.60,0.64,"1,87%","3,56%","6,78%","0,62%","0,59%",0.56,4.09,"-1,06%","-0,43%",6.06,True,"R$ 93,94",0.98,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True
120,KNHY11,Títulos e Val. Mob.,"R$ 106,18",132110.0,1.20,1.11,"3,06%","6,19%","11,38%","1,02%","1,03%",0.95,7.45,"-2,79%","-1,71%",3.65,True,"R$ 101,33",1.05,NaN,NaN,NaN,NaN,NaN,NaN,0,True,True,True


In [18]:
#df.to_excel('fundos_imobiliarios_.xlsx')
df.to_excel('fundos_imobiliarios.xlsx')

In [19]:
#df = pd.read_csv('/content/fundos_imobiliarios_.csv', sep=';', encoding='latin-1',decimal='.', thousands='.')
df = pd.read_excel('/content/fundos_imobiliarios.xlsx')
df

,Unnamed: 0,Codigofundo,Setor,Preço Atual,LiqDia,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (3M)Média,DY (6M)Média,DY12MM,DYAno,Variação Preço,Rentab.Período,RentabAcumulada,Patrimonio,VPA,PVPA,DYPatrimonial,VariaçãoPatrimonial,Rentab. Patr.no Período,Rentab. Patr.Acumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos,acceptable_pricing,diverssified_assets,razonably_occupied
0,37,CPFF11,Híbrido,"R$ 80,50",153960,0.80,0.98,"2,93%","5,74%","9,16%","0,98%","0,96%",0.76,5.74,"-3,24%","-2,29%",1.32,False,"R$ 80,68",1.00,NaN,NaN,NaN,NaN,NaN,NaN,0,True,1.0,True
1,39,CVBI11,Títulos e Val. Mob.,"R$ 103,80",246300,1.00,0.95,"2,85%","5,93%","12,40%","0,95%","0,99%",1.03,5.93,"0,10%","1,05%",2.47,True,"R$ 99,94",1.04,NaN,NaN,NaN,NaN,NaN,NaN,0,True,1.0,True
2,41,IRDM11,Títulos e Val. Mob.,"R$ 118,42",627740,0.97,0.75,"2,45%","5,16%","10,83%","0,82%","0,86%",0.90,5.16,"-1,30%","-0,56%",6.89,True,"R$ 97,26",1.22,NaN,NaN,NaN,NaN,NaN,NaN,0,False,1.0,True
3,42,RBRF11,Títulos e Val. Mob.,"R$ 80,50",269240,0.60,0.74,"2,07%","3,97%","8,21%","0,69%","0,66%",0.68,3.97,"-7,89%","-7,20%",15.11,True,"R$ 88,04",0.91,NaN,NaN,NaN,NaN,NaN,NaN,0,True,1.0,True
4,44,CPTS11,Títulos e Val. Mob.,"R$ 99,80",830660,1.00,0.99,"3,02%","6,06%","11,29%","1,01%","1,01%",0.94,6.06,"0,15%","1,14%",4.31,True,"R$ 95,35",1.05,NaN,NaN,NaN,NaN,NaN,NaN,0,True,1.0,True
5,45,VGIP11,Outros,"R$ 101,53",107080,0.95,0.93,"2,79%","5,61%","11,52%","0,93%","0,93%",0.96,5.61,"-3,32%","-2,42%",5.67,True,"R$ 95,68",1.06,NaN,NaN,NaN,NaN,NaN,NaN,0,True,1.0,True
6,48,MCCI11,Títulos e Val. Mob.,"R$ 102,30",258120,1.00,0.97,"2,53%","4,56%","8,08%","0,84%","0,76%",0.67,4.56,"0,00%","0,97%",4.60,True,"R$ 100,29",1.02,NaN,NaN,NaN,NaN,NaN,NaN,0,True,1.0,True
7,93,TRXF11,Outros,"R$ 102,48",144380,0.70,0.68,"2,02%","3,95%","7,46%","0,67%","0,66%",0.62,4.60,"-0,24%","0,43%",1.40,True,"R$ 106,31",0.96,NaN,NaN,NaN,NaN,0.00,NaN,44,True,1.0,True
8,115,HFOF11,Títulos e Val. Mob.,"R$ 91,75",178560,0.60,0.64,"1,87%","3,56%","6,78%","0,62%","0,59%",0.56,4.09,"-1,06%","-0,43%",6.06,True,"R$ 93,94",0.98,NaN,NaN,NaN,NaN,NaN,NaN,0,True,1.0,True
9,120,KNHY11,Títulos e Val. Mob.,"R$ 106,18",132110,1.20,1.11,"3,06%","6,19%","11,38%","1,02%","1,03%",0.95,7.45,"-2,79%","-1,71%",3.65,True,"R$ 101,33",1.05,NaN,NaN,NaN,NaN,NaN,NaN,0,True,1.0,True


In [20]:
df = df.rename(columns={'Códigodo fundo':'Codigofundo'
                        #'Preço Atual':'Preco',
                        #'Liquidez Diária':'Liq_dia',
                       # 'Dividendo':'Dividendo',
                        #'DividendYield':'DY'
                        #'DY (12M)Média':'DY(12M)Media',
                        #'DY (3M)Acumulado':'(DY_3M)',
                        #'DY (6M)Acumulado':'DY_(6M)',
                        ##'DY (12M)Acumulado':'DY(12M)',
                       # 'DY (3M)Média':'DY(3M)Media',
                        #'DY (6M)Média':'DY(6M)Media',
                        #'DY Ano':'DY_Ano',
                        #'PatrimônioLíq.':'Patrimonio'
                        #'QuantidadeAtivos': 'ativos'
#, 'PatrimônioLíq.':'Patr Liqu'
})

In [21]:
df.dtypes

Unnamed: 0                   int64
Codigofundo                 object
Setor                       object
Preço Atual                 object
LiqDia                       int64
Dividendo                  float64
DividendYield              float64
DY (3M)Acumulado            object
DY (6M)Acumulado            object
DY (12M)Acumulado           object
DY (3M)Média                object
DY (6M)Média                object
DY12MM                     float64
DYAno                      float64
Variação Preço              object
Rentab.Período              object
RentabAcumulada            float64
Patrimonio                    bool
VPA                         object
PVPA                       float64
DYPatrimonial               object
VariaçãoPatrimonial         object
Rentab. Patr.no Período     object
Rentab. Patr.Acumulada      object
VacânciaFísica             float64
VacânciaFinanceira          object
QuantidadeAtivos             int64
acceptable_pricing            bool
diverssified_assets 

In [22]:
symbols = df['Codigofundo']+'.SA'
symbols

0     CPFF11.SA
1     CVBI11.SA
2     IRDM11.SA
3     RBRF11.SA
4     CPTS11.SA
5     VGIP11.SA
6     MCCI11.SA
7     TRXF11.SA
8     HFOF11.SA
9     KNHY11.SA
10    KNIP11.SA
11    JSRE11.SA
12    VILG11.SA
13    VINO11.SA
14    HSML11.SA
15    TGAR11.SA
16    HABT11.SA
17    URPR11.SA
18    HGLG11.SA
19    HGCR11.SA
20    LVBI11.SA
21    MALL11.SA
22    XPCM11.SA
23    KNRI11.SA
24    MXRF11.SA
25    SARE11.SA
26    BRCO11.SA
27    XPSF11.SA
28    XPLG11.SA
29    XPCI11.SA
30    HGRU11.SA
31    HGRE11.SA
32    QAGR11.SA
33    GGRC11.SA
34    MGFF11.SA
35    BRCR11.SA
36    TORD11.SA
37    RBRR11.SA
38    HCTR11.SA
39    BTLG11.SA
40    BCFF11.SA
41    RBRP11.SA
42    RECT11.SA
43    RECR11.SA
Name: Codigofundo, dtype: object

In [23]:
start = datetime(2015, 1, 1)
end = datetime.today()

In [24]:
def pega_fundos(name):
  stock = yf.download(name, start = start, end = end)
  return stock

In [25]:
stock_dict = {}

for symbol in symbols:
  try:
    stock_dict[symbol] = pega_fundos(symbol)
  except:
    print('NÃO DEU CERTO PARA O TICKET',symbol)
print('TERMINOU')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [26]:
#stock_dict['ABCP11.SA']

In [27]:
returns = pd.DataFrame()

for key in stock_dict.keys():
  returns[key] = stock_dict[key]['Adj Close'].pct_change()

returns.head()

,CPFF11.SA,CVBI11.SA,IRDM11.SA,RBRF11.SA,CPTS11.SA,VGIP11.SA,MCCI11.SA,TRXF11.SA,HFOF11.SA,KNHY11.SA,KNIP11.SA,JSRE11.SA,VILG11.SA,VINO11.SA,HSML11.SA,TGAR11.SA,HABT11.SA,URPR11.SA,HGLG11.SA,HGCR11.SA,LVBI11.SA,MALL11.SA,XPCM11.SA,KNRI11.SA,MXRF11.SA,SARE11.SA,BRCO11.SA,XPSF11.SA,XPLG11.SA,XPCI11.SA,HGRU11.SA,HGRE11.SA,QAGR11.SA,GGRC11.SA,MGFF11.SA,BRCR11.SA,TORD11.SA,RBRR11.SA,HCTR11.SA,BTLG11.SA,BCFF11.SA,RBRP11.SA,RECT11.SA,RECR11.SA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-02-18,NaN,NaN,NaN,-0.008317,NaN,NaN,NaN,NaN,0.000000,0.004711,0.006237,-0.004149,0.004823,NaN,-0.001015,NaN,-0.012039,NaN,-0.013192,-0.001717,NaN,-0.006695,0.003361,-0.050724,-0.006162,NaN,NaN,NaN,NaN,NaN,-0.006403,-0.030017,-0.021538,0.000000,-0.022023,-0.019568,NaN,-0.010358,-0.010887,0.054653,NaN,-0.014177,0.004590,NaN
2020-02-19,-0.003895,0.014510,NaN,-0.014596,NaN,NaN,0.001348,-0.008621,0.000000,-0.011083,0.001868,0.000000,-0.006000,-0.018958,-0.000762,NaN,0.005539,NaN,-0.028240,0.000000,NaN,-0.014504,0.004187,-0.022523,-0.009743,0.005813,-0.003086,-0.011275,NaN,0.000775,0.001481,-0.002592,0.000000,-0.010737,-0.018105,-0.010044,NaN,0.003806,-0.013616,-0.014033,NaN,0.003345,0.003694,NaN
2020-02-20,0.013490,-0.014206,NaN,0.022913,NaN,NaN,0.000000,0.000000,-0.006312,-0.012241,0.002119,-0.001667,0.013038,0.034540,0.002542,NaN,0.000000,NaN,-0.011050,-0.002580,NaN,0.004588,0.038192,-0.000900,0.005367,-0.005779,-0.001548,0.016361,NaN,-0.002614,-0.002219,-0.008009,0.011056,-0.002171,0.027337,0.003143,NaN,0.007583,0.003802,0.004651,NaN,0.011111,0.000387,NaN
2020-02-21,-0.007041,0.007744,NaN,-0.030320,NaN,NaN,0.009615,-0.000087,0.003777,0.008029,0.000423,0.013773,-0.006515,0.005736,-0.004649,NaN,0.000000,NaN,-0.002793,0.001724,NaN,0.001810,-0.004819,0.023544,0.012456,-0.004926,0.000000,-0.014634,NaN,0.000000,-0.011045,-0.010907,-0.009950,0.005294,-0.000446,-0.019871,NaN,0.000000,-0.003623,0.022500,NaN,-0.003297,0.007842,NaN
2020-02-27,-0.019330,-0.007782,NaN,-0.046283,NaN,NaN,-0.011905,0.008609,-0.008038,-0.006926,-0.008200,-0.040510,-0.016234,-0.024569,-0.012314,NaN,-0.018644,NaN,-0.001625,-0.023064,NaN,-0.019355,-0.031477,-0.001408,-0.015817,-0.001287,0.003876,-0.010000,NaN,-0.022913,-0.027509,-0.033513,-0.006965,-0.019693,-0.035823,-0.010228,NaN,-0.010913,-0.007355,-0.031513,NaN,-0.029768,-0.010567,NaN


In [28]:
discount_dict = {}
def calcula_desconto(df):
  max_price = df['Adj Close'].describe()['75%']
  last_price = df['Adj Close'].iloc[-1]
  return ((last_price - max_price)/max_price)*100

In [29]:
for tick in stock_dict.keys():
  try:
    discount_dict[tick] =  calcula_desconto(stock_dict[tick])
  except:
    discount_dict[tick] = 0
discount_dict

{'BCFF11.SA': -1.6149870482651854,
 'BRCO11.SA': -13.956694043639818,
 'BRCR11.SA': -19.862843280505025,
 'BTLG11.SA': 6.115385202261118,
 'CPFF11.SA': -2.624892000736084,
 'CPTS11.SA': 0.0,
 'CVBI11.SA': -1.0391804216162446,
 'GGRC11.SA': -15.624327901692512,
 'HABT11.SA': 2.4267795195639383,
 'HCTR11.SA': -11.195805689671657,
 'HFOF11.SA': -13.000188890093817,
 'HGCR11.SA': -4.93087416969686,
 'HGLG11.SA': 0.6528846832362248,
 'HGRE11.SA': -11.2066650390625,
 'HGRU11.SA': -6.76880060720295,
 'HSML11.SA': -17.811020703390696,
 'IRDM11.SA': -13.47361958039666,
 'JSRE11.SA': -14.668608115075049,
 'KNHY11.SA': -7.428070505451724,
 'KNIP11.SA': -6.339586469360803,
 'KNRI11.SA': -14.810165484848625,
 'LVBI11.SA': -16.130624261543858,
 'MALL11.SA': -11.211269898034038,
 'MCCI11.SA': -0.669962934579737,
 'MGFF11.SA': -25.961706029694952,
 'MXRF11.SA': -4.919579177887603,
 'QAGR11.SA': -41.48020500354353,
 'RBRF11.SA': -21.768708643065864,
 'RBRP11.SA': -13.33152061848516,
 'RBRR11.SA': -5.08

In [30]:
def pega_setor(stock):
  return[x for x  in df[symbols == stock]['Setor']]
stock_info_df = pd.DataFrame.from_dict(discount_dict, orient='index').reset_index()
stock_info_df.columns = ['ticker','discount']
stock_info_df['sector'] = [pega_setor(tick)[0] for tick in stock_info_df['ticker']]

stock_info_df

,ticker,discount,sector
0,CPFF11.SA,-2.624892,Híbrido
1,CVBI11.SA,-1.039180,Títulos e Val. Mob.
2,IRDM11.SA,-13.473620,Títulos e Val. Mob.
3,RBRF11.SA,-21.768709,Títulos e Val. Mob.
4,CPTS11.SA,0.000000,Títulos e Val. Mob.
5,VGIP11.SA,-3.490885,Outros
6,MCCI11.SA,-0.669963,Títulos e Val. Mob.
7,TRXF11.SA,-3.370890,Outros
8,HFOF11.SA,-13.000189,Títulos e Val. Mob.
9,KNHY11.SA,-7.428071,Títulos e Val. Mob.


In [31]:
std_list = []

for tick in stock_info_df['ticker']:
  std_list.append(returns[tick].std()*100)

stock_info_df['risk'] = std_list 

stock_info_df.sort_values(by = 'discount').head(10)



,ticker,discount,sector,risk
22,XPCM11.SA,-59.833051,Lajes Corporativas,2.504979
32,QAGR11.SA,-41.480205,Outros,2.631758
34,MGFF11.SA,-25.961706,Títulos e Val. Mob.,1.576468
3,RBRF11.SA,-21.768709,Títulos e Val. Mob.,2.037279
42,RECT11.SA,-20.891088,Híbrido,1.269376
35,BRCR11.SA,-19.862843,Híbrido,1.145542
14,HSML11.SA,-17.811021,Shoppings,1.877265
25,SARE11.SA,-17.255296,Híbrido,1.931419
28,XPLG11.SA,-16.131062,Logística,0.960103
20,LVBI11.SA,-16.130624,Logística,1.506127


In [32]:
def pega_dividendos(stock):
  return[x for x  in df[symbols == stock]['DividendYield']]

stock_info_df['DividendYield'] = [pega_dividendos(tick)[0] for tick in stock_info_df['ticker']]
stock_info_df


,ticker,discount,sector,risk,DividendYield
0,CPFF11.SA,-2.624892,Híbrido,1.750572,0.98
1,CVBI11.SA,-1.039180,Títulos e Val. Mob.,1.537800,0.95
2,IRDM11.SA,-13.473620,Títulos e Val. Mob.,1.398240,0.75
3,RBRF11.SA,-21.768709,Títulos e Val. Mob.,2.037279,0.74
4,CPTS11.SA,0.000000,Títulos e Val. Mob.,NaN,0.99
5,VGIP11.SA,-3.490885,Outros,2.098968,0.93
6,MCCI11.SA,-0.669963,Títulos e Val. Mob.,1.522799,0.97
7,TRXF11.SA,-3.370890,Outros,1.480935,0.68
8,HFOF11.SA,-13.000189,Títulos e Val. Mob.,1.112874,0.64
9,KNHY11.SA,-7.428071,Títulos e Val. Mob.,1.413778,1.11


In [33]:
def Quantidade_Ativos(stock):
  return[x for x  in df[symbols == stock]['QuantidadeAtivos']]

stock_info_df['QuantidadeAtivos'] = [Quantidade_Ativos(tick)[0] for tick in stock_info_df['ticker']]

In [34]:
def Preco_atual(stock):
  return[x for x  in df[symbols == stock]['Preço Atual']]
stock_info_df['Preço Atual'] = [Preco_atual(tick)[0] for tick in stock_info_df['ticker']]

In [35]:
def DY12MM(stock):
  return[x for x  in df[symbols == stock]['DY12MM']]
stock_info_df['DY12MM'] = [DY12MM(tick)[0] for tick in stock_info_df['ticker']]

In [36]:
def Patrimonio(stock):
  return[x for x  in df[symbols == stock]['Patrimonio']]
stock_info_df['Patrimonio'] = [Patrimonio(tick)[0] for tick in stock_info_df['ticker']]

In [37]:
def P_VPA(stock):
  return[x for x  in df[symbols == stock]['PVPA']]
stock_info_df['PVPA'] = [P_VPA(tick)[0] for tick in stock_info_df['ticker']]

In [38]:
#def AGIO_P_VPA(stock):
#  return[x for x  in df[symbols == stock]['P/VPA']]
#stock_info_df['P/VPA'] = [AGIO_P_VPA(tick)[0] *100 -100 for tick in stock_info_df['ticker']]

In [39]:
def LiqDia(stock):
  return[x for x  in df[symbols == stock]['LiqDia']]
stock_info_df['LiqDia'] = [LiqDia(tick)[0] for tick in stock_info_df['ticker']]

In [40]:
def Dividendo(stock):
  return[x for x  in df[symbols == stock]['Dividendo']]
stock_info_df['Dividendo'] = [Dividendo(tick)[0] for tick in stock_info_df['ticker']]

In [41]:
def VacanciaFisica(stock):
  return[x for x  in df[symbols == stock]['VacânciaFísica']]
stock_info_df['VacânciaFísica'] = [VacanciaFisica(tick)[0] for tick in stock_info_df['ticker']]


In [42]:
import plotly.express as px
#stock_info_df = stock_info_df.dropna()

In [43]:
fig = px.scatter(stock_info_df, x = 'risk', y ='discount', color = 'sector', hover_name='ticker', log_x = True)
plotly.offline.plot(fig,filename='oportunidades.html')

'oportunidades.html'

In [44]:
#stock_info_df = stock_info_df.groupby(['ticker', 'sector']).agg({'PVPA':'nunique'}).reset_index()
#stock_info_df = stock_info_df.groupby(['sector']).apply(lambda x: x.sort_values(['sector'],ascending = False)).reset_index(drop = True)

#stock_info_df.groupby(['sector']).head(3)
#stock_info_df

In [45]:
#stock_info_df.groupby('sector').head(2)

In [46]:
#stock_info_df.sort_values(by='sector', ascending=True).groupby('sector').head(2)
#stock_info_df.sort_values(by='PVPA', ascending=True).head(10)

In [47]:
#stock_info_df.sort_values(by='PVPA', ascending=True).head(10).to_excel('fiis.xlsx')

In [48]:
#stock_info_df.to_excel('fiis.xlsx')
#DY 0,55% Mês PVPA = 0.95
stock_info_df.sort_values(by='DividendYield', ascending=False).sort_values(by='PVPA', ascending=True).sort_values(by='discount', ascending=False).sort_values(by='risk', ascending=False).groupby('sector').head(2).to_excel('fiis.xlsx')

In [49]:
stock_info_df

,ticker,discount,sector,risk,DividendYield,QuantidadeAtivos,Preço Atual,DY12MM,Patrimonio,PVPA,LiqDia,Dividendo,VacânciaFísica
0,CPFF11.SA,-2.624892,Híbrido,1.750572,0.98,0,"R$ 80,50",0.76,False,1.00,153960,0.80,NaN
1,CVBI11.SA,-1.039180,Títulos e Val. Mob.,1.537800,0.95,0,"R$ 103,80",1.03,True,1.04,246300,1.00,NaN
2,IRDM11.SA,-13.473620,Títulos e Val. Mob.,1.398240,0.75,0,"R$ 118,42",0.90,True,1.22,627740,0.97,NaN
3,RBRF11.SA,-21.768709,Títulos e Val. Mob.,2.037279,0.74,0,"R$ 80,50",0.68,True,0.91,269240,0.60,NaN
4,CPTS11.SA,0.000000,Títulos e Val. Mob.,NaN,0.99,0,"R$ 99,80",0.94,True,1.05,830660,1.00,NaN
5,VGIP11.SA,-3.490885,Outros,2.098968,0.93,0,"R$ 101,53",0.96,True,1.06,107080,0.95,NaN
6,MCCI11.SA,-0.669963,Títulos e Val. Mob.,1.522799,0.97,0,"R$ 102,30",0.67,True,1.02,258120,1.00,NaN
7,TRXF11.SA,-3.370890,Outros,1.480935,0.68,44,"R$ 102,48",0.62,True,0.96,144380,0.70,0.00
8,HFOF11.SA,-13.000189,Títulos e Val. Mob.,1.112874,0.64,0,"R$ 91,75",0.56,True,0.98,178560,0.60,NaN
9,KNHY11.SA,-7.428071,Títulos e Val. Mob.,1.413778,1.11,0,"R$ 106,18",0.95,True,1.05,132110,1.20,NaN


In [50]:
stock_info_df.sort_values(by='DividendYield', ascending=False).sort_values(by='PVPA', ascending=True).sort_values(by='discount', ascending=False).sort_values(by='risk', ascending=False).groupby('sector').head(3).describe()['DividendYield']['mean']

0.6794117647058824

In [51]:
stock_info_df.sort_values(by='DividendYield', ascending=False).sort_values(by='PVPA', ascending=True).sort_values(by='discount', ascending=False).sort_values(by='risk', ascending=False).groupby('sector').head(2)

,ticker,discount,sector,risk,DividendYield,QuantidadeAtivos,Preço Atual,DY12MM,Patrimonio,PVPA,LiqDia,Dividendo,VacânciaFísica
32,QAGR11.SA,-41.480205,Outros,2.631758,0.72,12,"R$ 51,00",0.60,False,0.85,108880,0.38,NaN
22,XPCM11.SA,-59.833051,Lajes Corporativas,2.504979,0.50,1,"R$ 36,20",1.29,False,0.51,125420,0.21,100.0
21,MALL11.SA,-11.211270,Shoppings,2.343178,0.54,7,"R$ 94,56",0.50,True,0.90,115350,0.53,3.0
5,VGIP11.SA,-3.490885,Outros,2.098968,0.93,0,"R$ 101,53",0.96,True,1.06,107080,0.95,NaN
13,VINO11.SA,-2.538317,Lajes Corporativas,2.082260,0.77,9,"R$ 59,13",0.68,True,0.99,244010,0.46,3.7
3,RBRF11.SA,-21.768709,Títulos e Val. Mob.,2.037279,0.74,0,"R$ 80,50",0.68,True,0.91,269240,0.60,NaN
25,SARE11.SA,-17.255296,Híbrido,1.931419,0.64,3,"R$ 82,00",0.46,True,0.82,240900,0.60,2.1
14,HSML11.SA,-17.811021,Shoppings,1.877265,0.58,5,"R$ 84,33",0.42,True,0.89,186730,0.50,6.7
18,HGLG11.SA,0.652885,Logística,1.864605,0.65,17,"R$ 166,50",0.62,True,1.14,386180,1.10,6.9
12,VILG11.SA,-10.432975,Logística,1.803224,0.53,15,"R$ 112,12",0.46,True,0.98,229030,0.61,0.4


In [52]:
stock_info_df.sort_values(by='DividendYield', ascending=False).sort_values(by='PVPA', ascending=True).sort_values(by='discount', ascending=False).sort_values(by='risk', ascending=False).groupby('sector').head(32).describe()['PVPA']['mean']

0.9690909090909089

In [53]:
#stock_info_df.sort_values(by='DividendYield', ascending=False).sort_values(by='PVPA', ascending=True).sort_values(by='discount', ascending=False).sort_values(by='risk', ascending=False).groupby('sector').head(2).to_excel('fiis.xlsx')
#stock_info_df.sort_values(by='DividendYield', ascending=False).sort_values(by='PVPA', ascending=True).sort_values(by='discount', ascending=False).sort_values(by='risk', ascending=False).groupby('sector').head(2)
#stock_info_df.sort_values(by=['DividendYield','PVPA','discount','risk'],axis = 0, ascending=False).groupby('sector').head(2)#.describe()['DividendYield']['mean']